In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [68]:
!pip3 install sentence-transformers

     |████████████████████████████████| 79 kB 5.7 MB/s 
     |████████████████████████████████| 4.0 MB 64.4 MB/s 
     |████████████████████████████████| 1.2 MB 68.7 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 6.5 MB 49.2 MB/s 
     |████████████████████████████████| 596 kB 73.0 MB/s 
     |████████████████████████████████| 895 kB 65.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=44797c4d6bbf91a9fc8b043eb90c95821e1029acc9fadefe0241435b607d1e3a
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [71]:
import numpy as np
import pandas as pd
import json
import os
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Parameters
task = "classify_conds"
experiment_type = 1
numrows = 0
augment_type = "none"

batch_size = 16
learning_rate = 3e-5
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
# Filename
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
filename

'task-classify_conds_exp-1_rows-0_agument-none'

In [5]:
# Save result
projpath = "/content/drive/MyDrive/UCB_MIDS/W266"

#saved_file = os.path.join(projpath, "saved_results",filename)

In [9]:
class tabulateResults:
  """ Tabulates results
  """
  def __init__(self, translate_file=None, agg_file=None, sel_file=None, conds_file=None):
    self.translate_file = translate_file
    self.agg_file = agg_file
    self.sel_file = sel_file
    self.conds_file = conds_file
    self.agg_exists = False
    self.sel_exists = False
    self.conds_exists = False
    self.dd = {"AGG": [""], "SEL": [""], "CONDS": [""], "EX": [""]}

    # read input files
    if self.translate_file != None:
      # set flags
      self.agg_exists, self.sel_exists, self.conds_exists = True, True, True
      # read json data
      self.trans_df = self.readfile(self.translate_file)
      # create AGG df
      self.agg_df = pd.DataFrame({})
      self.agg_df["pred_label"] = self.trans_df["pred_label"].apply(lambda x: self.extractAgg(x))
      self.agg_df["true_label"] = self.trans_df["true_label"].apply(lambda x: self.extractAgg(x))
      # create SEL df
      self.sel_df = pd.DataFrame({})
      self.sel_df["pred_label"] = self.trans_df["pred_label"].apply(lambda x: self.extractSel(x))
      self.sel_df["true_label"] = self.trans_df["true_label"].apply(lambda x: self.extractSel(x))
      # create CONDS df
      self.conds_df = pd.DataFrame({})
      self.conds_df["pred_label"] = self.trans_df["pred_label"].apply(lambda x: self.extractConds(x))
      self.conds_df["true_label"] = self.trans_df["true_label"].apply(lambda x: self.extractConds(x))
    else:
      if self.agg_file != None:
        self.agg_df = self.readfile(self.agg_file)
        self.agg_exists = True
      if self.sel_file != None:
        self.sel_df = self.readfile(self.sel_file)
        self.sel_exists = True
      if self.conds_file != None:
        self.conds_df = self.readfile(self.conds_file)
        self.conds_exists = True
    
    # AGG
    if self.agg_exists:
      # process each cond
      self.agg_df["true_label_proc"] = self.agg_df.true_label.apply(lambda x: self.processAggSel(x))
      self.agg_df["pred_label_proc"] = self.agg_df.pred_label.apply(lambda x: self.processAggSel(x))
      # evaluate match
      self.agg_df["match"] = self.agg_df.apply(lambda x: self.stringEqual(x["pred_label_proc"], x["true_label_proc"]), axis=1)
      # calculate accuracy
      self.dd["AGG"] = [round(100*self.agg_df["match"].mean(axis=0),1)]

    # SEL
    if self.sel_exists:
      # process each cond
      self.sel_df["true_label_proc"] = self.sel_df.true_label.apply(lambda x: self.processAggSel(x))
      self.sel_df["pred_label_proc"] = self.sel_df.pred_label.apply(lambda x: self.processAggSel(x))
      # evaluate match
      self.sel_df["match"] = self.sel_df.apply(lambda x: self.stringEqual(x["pred_label_proc"], x["true_label_proc"]), axis=1)
      # calculate accuracy
      self.dd["SEL"] = [round(100*self.sel_df["match"].mean(axis=0),1)]

    # CONDS
    if self.conds_exists:
      # process each cond
      self.conds_df["true_label_proc"] = self.conds_df.true_label.apply(lambda x: self.processConds(x))
      self.conds_df["pred_label_proc"] = self.conds_df.pred_label.apply(lambda x: self.processConds(x))
      # get condition count
      self.conds_df["true_numconds"] = self.conds_df.true_label_proc.apply(lambda x: self.countConds(x))
      self.conds_df["pred_numconds"] = self.conds_df.pred_label_proc.apply(lambda x: self.countConds(x))
      # evaluate match
      self.conds_df["match"] = self.conds_df.apply(lambda x: self.stringEqual(x["pred_label_proc"], x["true_label_proc"]), axis=1)
      # calculate accuracy    
      self.dd["CONDS"] = [round(100*self.conds_df["match"].mean(axis=0),1)]

    # calculate overall Execution accuracy
    if  self.agg_exists and self.sel_exists and self.conds_exists:
      # get each type's match
      a = np.array(self.agg_df["match"])
      s = np.array(self.sel_df["match"])
      c = np.array(self.conds_df["match"])
      # calculate overall EX match
      ex_match = a*s*c
      ex_match = round(100*ex_match.mean(),1)
      self.dd["EX"] = [ex_match]

    # generate final dataframe
    self.table = pd.DataFrame(self.dd)

  def readfile(self, file):
    """ Read input json file and take the second json element (the first one
        is only a header)
    """
    with open(file) as f:
      lines = f.readlines()
    count = 0
    for line in lines:
      d = json.loads(line.strip())
      if count == 0:
        pass
      else:
        df = pd.DataFrame(d)
      count += 1
    return df

  def extractAgg(self, txt):
    """ Extract AGG from pred and true labels of translate df 
    """
    pattern = r'\((.*?)\)'
    d = re.findall(pattern, txt)
    if d == []:
      agg = ""
    else:
      agg = d[0]
    return agg

  def extractSel(self, txt):
    """ Extract SEL from pred and true labels of translate df 
    """
    pattern = r'\[(.*?)\]'
    d = re.findall(pattern, txt)
    if d != []:
      sel = d[0]
    else:
      sel = ""
    return sel

  def extractConds(self, txt):
    """ Extract CONDS from pred and true labels of translate df 
    """
    pattern = r'\[(.*?)\]'
    d = re.findall(pattern, txt)
    conds = "".join(["["+n.replace("'", "")+"]" for n in d[2:]])
    return conds

  def processAggSel(self, txt):
    """ Strip, lowercase
    """
    return txt.strip().lower()

  def processConds(self, txt):
    """ Extract each where condition and post process it (strip, lowercase, sort for comparison).
    """
    pattern = r'\[(.*?)\]'
    extr_conds = re.findall(pattern, txt)
    out = [c.strip().lower() for c in extr_conds]
    out.sort()
    return out

  def countConds(self, condlist):
    """ Count where conditions
    """
    return len(condlist) 
  
  def stringEqual(self, txt1, txt2):
    """ Check two strings or lists if they are equal
    """
    if txt1 == txt2:
      out = 1
    else:
      out = 0
    return out


### Direct Translate Result

In [10]:
task = "translate"

experiment_type = 0
numrows = 0
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp0 = tabulateResults(translate_file=file)  

In [ ]:
experiment_type = 1
numrows = 0
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp1 = tabulateResults(translate_file=file)

In [ ]:
experiment_type = 2
numrows = 0
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp2 = tabulateResults(translate_file=file)

In [ ]:
experiment_type = 1
numrows = 0
augment_type = "column"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp1_aug = tabulateResults(translate_file=file)

In [ ]:
experiment_type = 0
numrows = 0
augment_type = "column"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp0_aug = tabulateResults(translate_file=file)

In [ ]:
experiment_type = 0
numrows = 0
augment_type = "synonym"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type+"1"
file = os.path.join(projpath, "saved_results",filename)
trans_exp0_aug_syn1 = tabulateResults(translate_file=file)

In [ ]:
experiment_type = 0
numrows = 0
augment_type = "synonym"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type+"2"
file = os.path.join(projpath, "saved_results",filename)
trans_exp0_aug_syn2 = tabulateResults(translate_file=file)

In [ ]:
experiment_type = 3
numrows = 1
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp3 = tabulateResults(translate_file=file)

In [ ]:
experiment_type = 1
numrows = 0
augment_type = "synonym"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type+"2"
file = os.path.join(projpath, "saved_results",filename)
trans_exp1_aug_syn2 = tabulateResults(translate_file=file)

In [ ]:
trans_exp3.table

,AGG,SEL,CONDS,EX
0,86.8,92.3,73.3,62.3


In [ ]:
finaltable_trans = pd.concat([trans_exp0.table, trans_exp1.table, trans_exp2.table, trans_exp3.table,
                              trans_exp1_aug.table, trans_exp0_aug.table, 
                              trans_exp0_aug_syn1.table, trans_exp0_aug_syn2.table, trans_exp1_aug_syn2.table])
finaltable_trans["Experiment"] = ["Standard", "+Schema", "+Schema ColTypes", "+Schema+ColTypes+Vals",
                                  "+Schema&Augmentation", "Augmentation", "Augmentation Synonym1", "Augmentation Synonym2",
                                  "+Schema&Augmentation Synonym2"]
finaltable_trans = finaltable_trans.set_index('Experiment')
finaltable_trans.style.set_properties(**{'text-align': 'left'})

,AGG,SEL,CONDS,EX
Experiment,,,,
Standard,84.200000,65.700000,46.000000,32.000000
+Schema,86.900000,93.300000,74.100000,63.100000
+Schema ColTypes,87.100000,92.900000,73.100000,62.200000
+Schema+ColTypes+Vals,86.800000,92.300000,73.300000,62.300000
+Schema&Augmentation,87.600000,93.800000,78.300000,66.800000
Augmentation,84.600000,69.000000,52.300000,37.600000
Augmentation Synonym1,84.100000,61.500000,43.700000,27.300000
Augmentation Synonym2,83.300000,57.200000,40.300000,25.500000
+Schema&Augmentation Synonym2,86.200000,90.300000,68.300000,56.300000


In [ ]:
# write csv
csv_file = os.path.join(projpath, "saved_results", "finaltable_trans.csv")
finaltable_trans.to_csv(csv_file)


### Slot Prediction Result

```
# This is formatted as code
```



In [ ]:
tasks = ["classify_agg", "classify_sel", "classify_conds"]

In [ ]:
experiment_type = 0
numrows = 0
augment_type = "none"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp0 = tabulateResults(agg_file=files[0], sel_file=files[1], conds_file=files[2])


In [ ]:
experiment_type = 1
numrows = 0
augment_type = "none"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp1 = tabulateResults(agg_file=files[0], sel_file=files[1], conds_file=files[2])


In [ ]:
experiment_type = 1
numrows = 0
augment_type = "column"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp1_aug = tabulateResults(agg_file=files[0], sel_file=files[1], conds_file=files[2])

In [ ]:
experiment_type = 1
numrows = 0
augment_type = "synonym"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp1_aug_syn = tabulateResults(agg_file=None, sel_file=None, conds_file=files[2])

In [ ]:
finaltable = pd.concat([exp0.table, exp1.table, exp1_aug.table, exp1_aug_syn.table])
finaltable["Experiment"] = ["Standard", "+Schema", "+Schema&AugColumn", "+Schema&AugSynonym"]
finaltable = finaltable.set_index('Experiment')
finaltable.style.set_properties(**{'text-align': 'left'})

,AGG,SEL,CONDS,EX
Experiment,,,,
Standard,88.900000,72.000000,45.400000,33.200000
+Schema,89.400000,92.100000,71.800000,60.500000
+Schema&AugColumn,90.000000,93.600000,76.200000,65.300000
+Schema&AugSynonym,,,65.900000,


In [ ]:
# write csv
csv_file = os.path.join(projpath, "saved_results", "finaltable.csv")
finaltable.to_csv(csv_file)


### Correction Logic

In [236]:
# Read db with embeddings
result_file = os.path.join(projpath, "test_emb_pickle")
dbfile = open(result_file, 'rb')     
db = pickle.load(dbfile)
dbfile.close()
# sentence transformer
from sentence_transformers import SentenceTransformer
st = SentenceTransformer('bert-base-nli-mean-tokens')

def correctLogic(table_id, col, val):
  """ Corrects column and its value based on closest embeddings distance
      to the ones in the database
  """
  # column
  col_enc = st.encode(col)
  header_emb = db[table_id]["header_emb"][0]
  header_sim = cosine_similarity([col_enc], header_emb)
  col_idx = np.argmax(header_sim)
  col_correct = db[table_id]["header"][col_idx]
  # value
  val_emb = np.array(db[table_id]["rows_emb"])[:,col_idx]
  val_enc = st.encode(q)
  val_sim = cosine_similarity([val_enc], val_emb)
  val_idx = np.argmax(val_sim)
  val_correct = np.array(db[table_id]["rows"])[val_idx,col_idx]
  return col_correct, val_correct

def processConds(txt, table_id, correction=False):
  """ Extract each where condition and post process it (strip, lowercase, sort for comparison).
  """
  table_id = table_id.strip()
  pattern = r'\[(.*?)\]'
  extr_conds = re.findall(pattern, txt)
  out = [c.strip().lower() for c in extr_conds]
  out.sort()
  if correction:
    out_correct = out
    for i in range(len(out_correct)):
      et = re.findall(r"(.+)equals to(.+)", out_correct[i])
      gt = re.findall(r"(.+)greater than(.+)", out_correct[i])
      lt = re.findall(r"(.+)less than(.+)", out_correct[i])
      correct_flag = False
      if len(et) != 0:
        if len(et[0]) == 2:
          op = "equals to"
          col = et[0][0].strip()
          val = et[0][1].strip()
          correct_flag = True
      elif len(gt) != 0:
        if len(gt[0]) == 2:
          op = "greater than"
          col = gt[0][0].strip()
          val = gt[0][1].strip()
          correct_flag = True     
      elif len(lt) != 0:
        if len(lt[0]) == 2:
          op = "less than"
          col = lt[0][0].strip()
          val = lt[0][1].strip()
          correct_flag = True   
      else:
        pass

      if correct_flag:
        col_correct, val_correct = correctLogic(table_id, col, val)
        cond_correct = col_correct + " " + op + " " + val_correct
        out_correct[i] = cond_correct.lower()      
    out = out_correct
  return out



In [237]:
pred_label = trans_exp0.conds_df["pred_label"][-5:]
table_id = table_id_df["table_id"][-5:]
true_label = trans_exp0.conds_df["true_label_proc"][-5:]
numcorrect = 0
for (i, (pr,id,tr)) in enumerate(zip(pred_label, table_id, true_label)):
  pred_correct = processConds(pr, id, correction=True)
  if pred_correct == tr:
    numcorrect += 1

acc_correct = numcorrect / len(pred_label)

